                                                    *ЗАДАЧА РЕГРЕССИИ*

In [1]:
from utils import path_setup
path_setup()
from src.paths import PROCESSED_DATA
import pandas as pd
import os




In [2]:
path=os.path.join(PROCESSED_DATA,"prepaired_data.csv")

df = pd.read_csv(path, index_col=0)
df

,Layer,Amp,FreQ,e,h,Optimal
0,4,6.0,2.0,12.2,0.9,1
1,5,6.0,1.5,12.6,2.0,1
2,6,6.0,1.0,14.1,1.7,0
4,8,5.0,2.0,13.6,1.3,0
5,9,5.0,1.6,11.8,1.6,0
...,...,...,...,...,...,...
69,7,4.9,1.8,NaN,NaN,0
70,4,5.4,1.7,NaN,NaN,1
71,5,5.4,1.7,NaN,NaN,1
72,6,5.4,1.7,NaN,NaN,0


In [3]:
df.describe()

,Layer,Amp,FreQ,e,h,Optimal
count,66.000000,66.000000,66.000000,49.000000,49.000000,66.000000
mean,6.272727,4.937879,1.746970,11.073469,1.659184,0.439394
std,1.957592,0.913281,0.244458,2.185442,0.565287,0.500117
min,4.000000,2.500000,1.000000,5.400000,0.400000,0.000000
25%,5.000000,4.500000,1.600000,10.400000,1.300000,0.000000
50%,6.000000,5.000000,1.800000,11.800000,1.600000,0.000000
75%,7.750000,5.500000,1.800000,12.800000,2.000000,1.000000
max,12.000000,6.500000,2.300000,14.100000,3.600000,1.000000


In [4]:
df = df.drop(columns=["Layer","Optimal"], errors="ignore")
df = df.dropna(axis=0)
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Amp,49.0,4.922449,1.033252,2.5,4.5,5.0,5.5,6.5
FreQ,49.0,1.736735,0.282617,1.0,1.6,1.8,2.0,2.3
e,49.0,11.073469,2.185442,5.4,10.4,11.8,12.8,14.1
h,49.0,1.659184,0.565287,0.4,1.3,1.6,2.0,3.6


In [5]:
from sklearn.model_selection import train_test_split


y=df["e"]
features = ["Amp","FreQ"]
X = df[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.8, 
                                                    test_size=0.2, 
                                                    random_state=13)






In [8]:
!pip install "setuptools<81"

In [9]:

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestRegressor
from src.modeling.pipe_construct import PipeConstruct


loo = LeaveOneOut() # Назначение метода кросс-валидации

# Пайплайн с LinearRegressor
pipe_linreg = PipeConstruct(
    set_scaler=StandardScaler(),
    set_model=LinearRegression()
)

# Пайплайн с RandomForestRegressor
pipe_rf = PipeConstruct(
    set_scaler=StandardScaler(),
    set_model=RandomForestRegressor(n_estimators=7, random_state=0)
)

print("Результат для линейной регрессии:\n")
pipe_linreg.cross_validation(X_train,y_train,cv=loo)
pipe_linreg.fit(X_train, y_train)
pipe_linreg.error_score(X_test,y_test)

print("Результат для случайного леса:\n")
pipe_rf.cross_validation(X_train,y_train,cv=loo)
pipe_rf.fit(X_train, y_train)
pipe_rf.error_score(X_test,y_test)


Результат для линейной регрессии:



/opt/homebrew/Cellar/python@3.12/3.12.8/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/opt/homebrew/Cellar/python@3.12/3.12.8/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/opt/homebrew/Cellar/python@3.12/3.12.8/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See http

Cross-validation Average MAE: 0.66

MAE: 0.63
RMSE: 0.75
Коэффициент детерминации: 0.90

Результат для случайного леса:

Cross-validation Average MAE: 0.76

MAE: 0.44
RMSE: 0.82
Коэффициент детерминации: 0.88



In [ ]:
from src.modeling.save_model import save_model

save_model(pipe_linreg,"regression.pkl")